In [ ]:
 import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving spam_ham_dataset.csv to spam_ham_dataset.csv


In [ ]:
df = pd.read_csv('spam_ham_dataset.csv')
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [ ]:
df.shape
df.columns

Index(['Unnamed: 0', 'label', 'text', 'label_num'], dtype='object')

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

(5171, 4)

In [ ]:
df.isnull().sum()

Unnamed: 0    0
label         0
text          0
label_num     0
dtype: int64

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def process_text(text):
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)

  clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
  return clean_words

In [ ]:
df['text'].head().apply(process_text)

0    [Subject, enron, methanol, meter, 988291, foll...
1    [Subject, hpl, nom, january, 9, 2001, see, att...
2    [Subject, neon, retreat, ho, ho, ho, around, w...
3    [Subject, photoshop, windows, office, cheap, m...
4    [Subject, indian, springs, deal, book, teco, p...
Name: text, dtype: object

In [ ]:
#convert a collection of text to a matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer
message_bow = CountVectorizer(analyzer=process_text).fit_transform(df['text'])

In [ ]:
#split the data into 80% training and testing

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(message_bow,df['label_num'],test_size=0.2,random_state=0)

In [ ]:
#Get the shape of messages_bow
message_bow.shape

(5171, 50381)

In [ ]:
#create and train Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB().fit(x_train,y_train)

In [ ]:
#print the predictions

print(classifier.predict(x_train))

#print the actual values
print(y_train)

[0 0 0 ... 1 0 0]
3628    0
2491    0
3262    0
2972    0
2481    1
       ..
4931    1
3264    1
1653    1
2607    0
2732    0
Name: label_num, Length: 4136, dtype: int64


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

pred = classifier.predict(x_train)
print(classification_report(y_train, pred))
print()
print('confusion matrix: ',confusion_matrix(y_train, pred))
print()
print('Accuracy: ',accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2940
           1       0.98      0.97      0.98      1196

    accuracy                           0.99      4136
   macro avg       0.99      0.98      0.98      4136
weighted avg       0.99      0.99      0.99      4136


confusion matrix:  [[2918   22]
 [  30 1166]]

Accuracy:  0.9874274661508704


In [ ]:
#print the predictions
print(classifier.predict(x_test))

#print the actual values
print(y_test.values)

[0 0 0 ... 0 1 0]
[0 0 0 ... 0 1 0]


In [ ]:
pred = classifier.predict(x_test)
print(classification_report(y_test, pred))
print()
print('confusion matrix: ',confusion_matrix(y_test, pred))
print()
print('Accuracy: ',accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       732
           1       0.95      0.96      0.96       303

    accuracy                           0.97      1035
   macro avg       0.97      0.97      0.97      1035
weighted avg       0.97      0.97      0.97      1035


confusion matrix:  [[718  14]
 [ 13 290]]

Accuracy:  0.9739130434782609


In [ ]:
import pickle

with open('spam_email_detection_model',"wb") as file:
    pickle.dump(classifier, file)


In [ ]:
!mkdir -p saved_model

In [ ]:
import pickle

with open('saved_model/spam_email_detection_model',"wb") as file:
    pickle.dump(classifier, file)

In [ ]:
!ls

sample_data  saved_model  spam_email_detection_model  spam_ham_dataset.csv
